In [70]:
import gzip
import pandas as pd
import numpy as np
from itertools import groupby
from sklearn.cluster import DBSCAN


In [71]:
# Read and filte interchroms interactions
def read_and_filter_data(file_path):

    columns = ['str1','chr1', 'pos1', 'frag1', 'str2', 'chr2', 'pos2', 
               'frag2', 'mapq1', 'cigar1', 'sequence1', 'mapq2', 
               'cigar2', 'sequence2', 'readname1', 'readname2']
    
    # Open and read the gzip file
    with gzip.open(file_path, 'rt') as f:
        data = pd.read_csv(f, delimiter=' ', header=None, names=columns, index_col=False)
        data = data[data['chr1'] != data['chr2']].copy()
    
    return data

In [72]:
# Extract chrom lengths
def extract_chromosome_lengths(file_path):
    chromosome_lengths = {}
    
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip comments
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 9:
                chr_name = fields[9]  # UCSC-style name (chr1, chr2, etc.)
                chr_length = int(fields[8])  # Sequence-Length
                chromosome_lengths[chr_name] = chr_length
    
    return chromosome_lengths

In [73]:
# Create bin df with bin coordinates and chrrom names
def create_bins(data, binsize, chromosome_lengths):

    bins = []
    bin_index = 0
    
    for chrom, chrom_length in chromosome_lengths.items():
        # Reset bin_start for each chromosome
        bin_start = 1
        while bin_start <= chrom_length:
            bin_end = min(bin_start + binsize - 1, chrom_length)
            bins.append((chrom, bin_start, bin_end, bin_index))
            bin_start = bin_end + 1
            bin_index += 1
    
    # Convert bins to a DataFrame
    bins_df = pd.DataFrame(bins, columns=['chrom', 'bin_start', 'bin_end', 'bin_index'])
    
    return bins_df

In [119]:
# Add bin indexes to df with all information
def assign_bins_to_interactions(data, bins_df):
    # Create a dictionary for fast lookup of bin indexes based on chromosome and position
    bins_dict = {}
    
    for _, row in bins_df.iterrows():
        chrom = row['chrom']
        bin_start = row['bin_start']
        bin_end = row['bin_end']
        bin_index = row['bin_index']
        
        if chrom not in bins_dict:
            bins_dict[chrom] = []
        bins_dict[chrom].append((bin_start, bin_end, bin_index))
    
    def find_bin_index(chrom, pos):
        if chrom in bins_dict:
            for bin_start, bin_end, bin_index in bins_dict[chrom]:
                if bin_start <= pos <= bin_end:
                    return int(bin_index)
        return -1

    data['bin1_index'] = data.apply(lambda row: find_bin_index(row['chr1'], row['pos1']), axis=1)
    data['bin2_index'] = data.apply(lambda row: find_bin_index(row['chr2'], row['pos2']), axis=1)
    
    return data

In [89]:
def create_symmetric_matrix(bin_df, data_with_bins):
    # Initialize an empty square matrix of size len(bin_df) x len(bin_df)
    matrix_size = len(bin_df) 
    symmetric_matrix = np.zeros((matrix_size, matrix_size), dtype=int)

    # Iterate over the data_with_bins to populate the symmetric matrix
    for _, row in data_with_bins.iterrows():
        bin1 = row['bin1_index']
        bin2 = row['bin2_index']
        
        # Add the value to both (bin1, bin2) and (bin2, bin1) to ensure symmetry
        symmetric_matrix[bin1, bin2] += 1
        symmetric_matrix[bin2, bin1] += 1
    
    # Convert the numpy array to a DataFrame for better readability
    symmetric_matrix_df = pd.DataFrame(symmetric_matrix, index=bin_df['bin_index'], columns=bin_df['bin_index'])
    symmetric_matrix_df.index.name = 'bin1_index'
    symmetric_matrix_df.columns.name = 'bin2_index'
    
    return symmetric_matrix_df

In [96]:
def normalize_contact_matrix(contact_matrix, bins_df, chromosome_lengths):
    bin_to_chrom = bins_df.set_index('bin_index')['chrom'].to_dict()
    
    df_long = contact_matrix.stack().reset_index()
    df_long.columns = ['bin1_index', 'bin2_index', 'value']

    df_long['chrom1'] = df_long['bin1_index'].map(bin_to_chrom)
    df_long['chrom2'] = df_long['bin2_index'].map(bin_to_chrom)
    
    df_long['normalization_factor'] = df_long['chrom1'].map(chromosome_lengths) * df_long['chrom2'].map(chromosome_lengths)
    df_long['normalized_value'] = df_long['value'] / df_long['normalization_factor']

    df_long = df_long.drop(columns=['value', 'chrom1', 'chrom2', 'normalization_factor'])

    df_long = df_long.groupby(['bin1_index', 'bin2_index'])['normalized_value'].mean().reset_index()

    try:
        normalized_matrix = df_long.pivot(index='bin1_index', columns='bin2_index', values='normalized_value').fillna(0)
    except Exception as e:
        print("Ошибка при преобразовании в pivot:", str(e))
        return df_long  # Возвращаем в длинном формате, если ошибка

    # Устанавливаем правильные имена индексов и столбцов
    normalized_matrix.index.name = 'bin1_index'
    normalized_matrix.columns.name = 'bin2_index'

    normalized_matrix.index.name = 'bin1_index'
    normalized_matrix.columns.name = 'bin2_index'
    
    return normalized_matrix

In [76]:
# Look for pair of beans with high values in matrix
def detect_high_values(matrix, percentile=0):
    high_value_threshold = np.percentile(matrix.values.flatten(), percentile)

    high_value_bins = matrix[matrix > high_value_threshold].stack().reset_index()
    high_value_bins.columns = ['bin1', 'bin2', 'value']
    
    return high_value_threshold, high_value_bins

In [77]:
# Cluster windows with high values
def cluster_and_select_best_region(coords_list, eps=1.5, min_samples=2):

    if len(coords_list) == 0:
        return []

    # Convert list of coordinates to numpy array
    coords_array = np.array(coords_list)
    
    # Apply DBSCAN clustering
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(coords_array)
    
    # Cluster labels (-1 is for noise)
    labels = clustering.labels_
    
    # Initialize a list to store the best region from each cluster
    best_regions = []
    
    # Iterate over unique clusters
    for label in set(labels):
        if label == -1:
            continue  # Skip noise points
        
        # Get indices of regions in this cluster
        cluster_indices = np.where(labels == label)[0]
        cluster_coords = [coords_list[i] for i in cluster_indices]
        
        # Determine the "best" region in the cluster (by max sum)
        best_coords = cluster_coords[0]  # Placeholder, you can improve selection criteria
        best_regions.append(best_coords)
    
    return best_regions


In [145]:
# Detect translocation regions
def find_top_balanced_high_contact_regions(matrix, region_size=10, threshold=20, top_n=10, tolerance=0.1):
    region_sums = []
    visited_coords = set()  
    
    for i in range(matrix.shape[0] - region_size + 1):
        for j in range(matrix.shape[1] - region_size + 1):
            if (i, j) in visited_coords or (j, i) in visited_coords:
                continue
            
            region = matrix[i:i+region_size, j:j+region_size]
            region_filtered = region * (region > threshold)
            
            row_sums = np.sum(region_filtered, axis=1)
            col_sums = np.sum(region_filtered, axis=0)
            
            row_sum_mean = np.mean(row_sums)
            col_sum_mean = np.mean(col_sums)
            
            if np.all(np.abs(row_sums - row_sum_mean) / row_sum_mean < tolerance) and \
               np.all(np.abs(col_sums - col_sum_mean) / col_sum_mean < tolerance):
                region_sum = np.sum(region_filtered)
                region_sums.append(((i, j), region_sum, region))
                
                visited_coords.add((i, j))
                visited_coords.add((j, i))
    
    top_regions = sorted(region_sums, key=lambda x: x[1], reverse=True)#[:top_n]
    
    # Cluster and select the best region
    cluster_coords = cluster_and_select_best_region([coords for coords, region_sum, region in top_regions], eps=2, min_samples=5)
    
    # Map the clusters to top regions
    top_regions_filtered = [region for region in top_regions if region[0] in cluster_coords]

    return [(coords, region_sum) for coords, region_sum, region in top_regions_filtered]


In [214]:
def filter_true_regions(region_list, threshold_factor=2, percentile=90):
    if not region_list:
        return []
    
    region_sums = np.array([sum_val for _, sum_val in region_list])
    mean_val = np.mean(region_sums)
    std_dev = np.std(region_sums)
    
    # Адаптивный threshold_factor на основе размера выборки
    if len(region_sums) > 50:
        threshold_factor = max(threshold_factor, np.log10(len(region_sums)))
    else:
        threshold_factor = threshold_factor
    
    potential_true_regions = []
    for i in range(len(region_sums)):
        sum_val = region_sums[i]
        if sum_val > mean_val + threshold_factor * std_dev:
            if i > 0:
                previous_sum = region_sums[i-1]
                difference = abs(previous_sum - sum_val)
                if difference / previous_sum < 0.1:  # Пример порога для резкой ступени
                    potential_true_regions.append((region_list[i][0], sum_val))
            else:
                potential_true_regions.append((region_list[i][0], sum_val))
    
    if not potential_true_regions:
        return [(region, sum_val, False) for region, sum_val in region_list]
    
    percentile_value = np.percentile(region_sums, percentile)
    
    true_regions = [(region, sum_val) for region, sum_val in potential_true_regions if sum_val >= percentile_value]
    true_regions_set = set([region for region, _ in true_regions])
    
    marked_regions = [(region, sum_val, region in true_regions_set) for region, sum_val in region_list]
    
    return marked_regions

In [189]:
def map_coords_to_chromosome(coords_list, bin1_df):

    mapped_results = []
    
    for coords, region_sum, b in coords_list:
        chrom1 = bin1_df.loc[bin1_df['bin_index'] == coords[0], 'chrom'].values[0]
        chrom2 = bin1_df.loc[bin1_df['bin_index'] == coords[1], 'chrom'].values[0]
        
        mapped_results.append(((chrom1, chrom2), region_sum, b))
    
    return mapped_results

In [216]:
merged_nodups_file = '/media/eternus1/nfs/projects/dpanchenko/test_embrio/data/merged_nudups/Kaz3_K_BGI_merged_nodups.txt.gz'
chromosome_lengths_file = '/media/eternus1/nfs/projects/dpanchenko/test_embrio/data/genome/GCF_009914755.1_T2T-CHM13v2.0_assembly_report.txt'

chromosome_lengths = extract_chromosome_lengths(chromosome_lengths_file)
data = read_and_filter_data(merged_nodups_file)

binsize = 5000000
bins_df = create_bins(data, binsize, chromosome_lengths)
data_with_bins = assign_bins_to_interactions(data, bins_df)

contact_matrix = create_symmetric_matrix(bins_df, data_with_bins)
# normalized_matrix = normalize_contact_matrix(contact_matrix, bins_df, chromosome_lengths)

In [200]:
# high_value_threshold, high_value_bins = detect_high_values(contact_matrix, percentile=90)


In [217]:
top_balanced_regions = find_top_balanced_high_contact_regions(contact_matrix.values, region_size=10, threshold=0, top_n=10000, tolerance=0.99)


/tmp/ipykernel_2545017/3559144135.py:20: RuntimeWarning: invalid value encountered in divide
  if np.all(np.abs(row_sums - row_sum_mean) / row_sum_mean < tolerance) and \


In [ ]:
[(coords, region_sum) for coords, region_sum in top_balanced_regions]

In [218]:
true_regions = filter_true_regions([(coords, region_sum) for coords, region_sum in top_balanced_regions], threshold_factor=2, percentile=90)
mapped_chromosomes = map_coords_to_chromosome(true_regions, bins_df)

mapped_chromosomes

[(('chr17', 'chr19'), 577, True),
 (('chr19', 'chr22'), 489, False),
 (('chr15', 'chr17'), 486, True),
 (('chr1', 'chr19'), 474, True),
 (('chr17', 'chr22'), 462, True),
 (('chr1', 'chr17'), 452, True),
 (('chr19', 'chr22'), 400, False),
 (('chr1', 'chr15'), 391, True),
 (('chr9', 'chr19'), 372, False),
 (('chr1', 'chr10'), 354, False),
 (('chr19', 'chr20'), 322, False),
 (('chr12', 'chr19'), 316, False),
 (('chr10', 'chr19'), 307, False),
 (('chr14', 'chr16'), 306, False),
 (('chr12', 'chr15'), 301, False),
 (('chr14', 'chr22'), 301, False),
 (('chr10', 'chr17'), 299, False),
 (('chr9', 'chr16'), 296, False),
 (('chr1', 'chr9'), 291, False),
 (('chr4', 'chr8'), 288, False),
 (('chr5', 'chr15'), 286, False),
 (('chr9', 'chr22'), 285, False),
 (('chr1', 'chr16'), 282, False),
 (('chr1', 'chr5'), 276, False),
 (('chr12', 'chr15'), 276, False),
 (('chr2', 'chr7'), 271, False),
 (('chr9', 'chr15'), 267, False),
 (('chr3', 'chr15'), 261, False),
 (('chr12', 'chr19'), 257, False),
 (('chr9',

In [196]:
region_sums = np.array([sum_val for _, sum_val in top_balanced_regions])
mean_val = np.mean(region_sums)
std_dev = np.std(region_sums)
potential_true_regions = [(region, sum_val) for region, sum_val in top_balanced_regions if sum_val > mean_val + 2 * std_dev]

percentile_value = np.percentile(region_sums, 90)
[(region, sum_val) for region, sum_val in potential_true_regions if sum_val >= percentile_value]

[((511, 545), 577),
 ((547, 585), 489),
 ((476, 511), 486),
 ((0, 545), 474),
 ((511, 585), 462),
 ((0, 518), 452),
 ((552, 583), 400),
 ((1, 486), 391),
 ((334, 546), 372),
 ((0, 364), 354)]

In [179]:
potential_true_regions

[((55, 512), 566), ((486, 546), 554)]

In [168]:
std_dev

143.55641554609966